In [1]:
import csv
import nni
import time
import json
import math
import copy
import argparse
import warnings
import numpy as np
import esm
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import gc
import pandas as pd
from utils import *
from model import *
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling
from tqdm import tqdm
import re
from sklearn.metrics import f1_score
from dataprocess import *
import random
import os
import json
import random
import itertools
from collections import defaultdict
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
import pandas as pd
from io import StringIO
import warnings
warnings.filterwarnings("ignore")


/data/dengrui/anaconda3/envs/micro-ppi/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
test_data = pd.read_csv('../data/example_test_data.csv')

### Load configuration

In [3]:
param,args = get_param()

### load test data

In [4]:
test_data = pd.read_csv('../data/example_test_data.csv')
test_data

,item_id_a,seq_A,pdb_A,item_id_b,seq_B,pdb_B
0,Q9LZP9,MATIATGLNIATQRVFVTSENRPVCLAGPVHLNNSWNLGSRTTNRM...,AF-Q9LZP9-F1-model_v4.pdb,Q8L960,MASVAASSSISFAASFLKIKAFPLSPRFFPIRTLRCSVSSSSSEPI...,AF-Q8L960-F1-model_v4.pdb
1,Q9MA79,MDHAADAHRTDLMTITRFVLNEQSKYPESRGDFTILLSHIVLGCKF...,AF-Q9MA79-F1-model_v4.pdb,Q96514,MSILLCFLCLLPVFLVSLSILSKRLKPSKWKLPPGPKTLPIIGNLH...,AF-Q96514-F1-model_v4.pdb
2,Q9ZU52,MASASFVKPNTLSSPWIGQRSFAHTSASSSPPPRVSFAIRAGAYSD...,AF-Q9ZU52-F1-model_v4.pdb,Q24JN5,MASKSKQHLRYQPRKSVSRSTQAFTVLILLLVVILILLGLGILSLP...,AF-Q24JN5-F1-model_v4.pdb
3,Q9SAJ4,MATKRSVGTLKEADLKGKSVFVRVDLNVPLDDNSNITDDTRIRAAV...,AF-Q9SAJ4-F1-model_v4.pdb,Q9SIH1,MSARPEGSPPEVTLETSMGPFTVEMYYKHSPRTCRNFLELSRRGYY...,AF-Q9SIH1-F1-model_v4.pdb
4,O22914,MTTIAAAGLNVATPRVVVRPVARVLGPVRLNYPWKFGSMKRMVVVK...,AF-O22914-F1-model_v4.pdb,Q8LA13,MSASWADVADSEKAVSQSKPPYVPPHLRNRPSEPVAAPLPQNDHAG...,AF-Q8LA13-F1-model_v4.pdb


In [5]:
base_tag = "interaction_test_res"
path = '/data/dengrui/workbase/Micro-PPI/data/temp/'

### load model

In [6]:
model_ckpt_path = '../model/Micro-PPI.pth'
model = GIN(param).to(device)
model.load_state_dict(torch.load(model_ckpt_path, map_location=device))

<All keys matched successfully>

In [ ]:
# For large datasets, you can run predictions in batches.
total_n = len(test_data)
sample_n = total_n 
effective_runs = math.ceil(total_n / sample_n)  
perm = np.random.permutation(total_n)
base_tag = "interaction_test_res"
output_csv_path = f"../results/predictions_{base_tag}_all_runs.csv"


In [8]:
all_run_dfs = []

for run_idx in range(effective_runs):
    print(f'Processing run_idx: {run_idx} / {effective_runs}')
    
    start = run_idx * sample_n
    end = min(start + sample_n, total_n)
    if start >= total_n:
        break

    sel_pos = perm[start:end]
    if isinstance(test_data, (pd.DataFrame, pd.Series, pd.Index)):
        subset = test_data.iloc[sel_pos]
    else:
        subset = np.asarray(test_data)[sel_pos]

    dataset_tag = f"all_test_{base_tag}_run{run_idx}_20251217"
    esm_out_fname = f"{path}embedding_esm2_1024_interaction_test_{dataset_tag}.npy"
     # 获取数据
    protein_name, protein_data, ppi_g, ppi_list, labels, ppi_split_dict = get_ppi_label(
        subset, dataset_tag, args.split_mode, args.seed, path, 'test'
    )

    vae_model = CodeBook(
        param,
        DataLoader(protein_data, batch_size=256, shuffle=False, collate_fn=collate)
    ).to(device)
    
    args.ckpt_path = '../model/Stru_embedding.ckpt'
    vae_model.load_state_dict(torch.load(args.ckpt_path, map_location=device))
    
    # 获取 Embedding
    prot_embed = vae_model.Protein_Encoder.forward(vae_model.vq_layer).to(device)
    string_embedding_esm2 = get_esm_embedding(subset, protein_name, esm_out_fname)
    string_embedding_esm2_tensor = torch.tensor(string_embedding_esm2).to(device) # 确保也在 device

    train_pred, train_tru, eval_edges_train, eval_prob_train, _ = evaluator(
        model, ppi_g, prot_embed, string_embedding_esm2_tensor, ppi_list, labels,
        ppi_split_dict['train_index'], 2048, 'output'
    )
    test_pred, test_tru, eval_edges_test, eval_prob_test, _ = evaluator(
        model, ppi_g, prot_embed, string_embedding_esm2_tensor, ppi_list, labels,
        ppi_split_dict['test_index'], 2048, 'output'
    )
    val_pred, val_tru, eval_edges_val, eval_prob_val, _ = evaluator(
        model, ppi_g, prot_embed, string_embedding_esm2_tensor, ppi_list, labels,
        ppi_split_dict['val_index'], 2048, 'output'
    )

    # 数据转换辅助函数
    def to_numpy(x):
        if torch.is_tensor(x):
            return x.detach().cpu().numpy()
        return np.asarray(x)

    # 拼接数据
    all_preds = np.concatenate([to_numpy(train_pred), to_numpy(val_pred), to_numpy(test_pred)], axis=0)
    all_eval_edges = np.concatenate([to_numpy(eval_edges_train), to_numpy(eval_edges_val), to_numpy(eval_edges_test)], axis=0)
    all_eval_probs = np.concatenate([to_numpy(eval_prob_train), to_numpy(eval_prob_val), to_numpy(eval_prob_test)], axis=0)

    # 压缩维度
    def squeeze_to_1d_preds(a):
        if a.ndim == 1: return a
        if a.ndim == 2 and a.shape[1] == 1: return a[:, 0]
        if a.ndim == 2 and a.shape[1] > 1:  return a.argmax(axis=1)
        raise ValueError(f"Unsupported prediction shape: {a.shape}")

    pred_1d = squeeze_to_1d_preds(all_preds)
    
    # 处理 Edge 形状
    if all_eval_edges.ndim != 2 or all_eval_edges.shape[1] != 2:
        all_eval_edges = all_eval_edges.reshape(-1, 2)

    n = all_eval_edges.shape[0]
    
    # ID 映射
    protein_a_index = all_eval_edges[:, 0].tolist()
    protein_b_index = all_eval_edges[:, 1].tolist()
    value_to_key = {v: k for k, v in protein_name.items()}
    protein_A_ids = [value_to_key[a] for a in protein_a_index]
    protein_B_ids = [value_to_key[b] for b in protein_b_index]

    # 构建当前批次的 DataFrame
    df_batch = pd.DataFrame({
        'run_idx': [run_idx] * n,
        'dataset_tag': [dataset_tag] * n,
        'Protein_A_Index': protein_a_index,
        'Protein_B_Index': protein_b_index,
        'protein_A_ids': protein_A_ids,
        'protein_B_ids': protein_B_ids,
        'Prediction': pred_1d,
        'probs': all_eval_probs
    })

    df_batch.to_csv(output_csv_path, mode='a', header=(run_idx == 0), index=False)
    # 删除占用显存的大对象
    del ppi_g
    del prot_embed
    del string_embedding_esm2_tensor
    del vae_model  
    
    del df_batch
    del subset
    del protein_data
    del string_embedding_esm2
    del all_preds, all_eval_edges, all_eval_probs
    
    gc.collect()
    torch.cuda.empty_cache()

print(f"Done. Result saved to {output_csv_path}")


Processing run_idx: 0 / 1
Files already exist. Loading the existing files...
Done. Result saved to ../results/predictions_interaction_test_res_all_runs_copy_20251224.csv
